# High throughput calculations

## Singlepoint

In [ ]:
from aiida import load_profile
load_profile()

We need to choose a model and architecture to be used for the calculation and save it as ModelData type, a specific data type of this plugin.
In this example we use MACE with a model that we download from this URL: "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model", and we save the file in the cache folder (default="~/.cache/mlips/"):


In [ ]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

If we already have the model saved in some folder we can save it as:

In [ ]:
# model = ModelData.from_local("/path/to/model", architecture="mace")

Another parameter that we need to define as AiiDA type is the code. Assuming the code is saved as `janus` in the `localhost` computer, the code info that are needed can be loaded as follow:


In [ ]:
from aiida.orm import load_code
code = load_code("janus@localhost")

Inputs should include the model, code, metadata, and any other keyword arguments expected by the calculation we are running:

In [ ]:
inputs = {
    "model": model,
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    "code": code,
    "properties": "energy",
}

We must also choose the calculation to perform:

In [ ]:
from aiida.plugins import CalculationFactory
SinglepointCalc = CalculationFactory("mlip.sp")

Then we get create our WorkGraph. This requires setting the calculation to be performed, the folder containing the files run the calculations on, inputs expected by the calculation, and the key to access the final structure from the calculations:

In [ ]:
from pathlib import Path

from aiida_mlip.workflows.ht_workgraph import get_ht_workgraph

wg = get_ht_workgraph(
    calc=SinglepointCalc,
    folder=Path("../../tests/workflows/structures/"),
    calc_inputs=inputs,
    final_struct_key="xyz_output",
)

In [ ]:
wg

Now we can run the calculations:

In [ ]:
wg.run()

The full graph can be visualised:

In [ ]:
from aiida_workgraph.utils import generate_node_graph

generate_node_graph(wg.pk)

If using `wg.submit()` instead of `wg.run()`, calculations may still running, even when setting `wait=True`, due to a timeout. The WorkGraph's state can be updated:

In [ ]:
wg.update()
wg.state

The final structures can then be accessed:

In [ ]:
wg.process.outputs.final_structures.H2O

## Geometry optimisation

Similarly, high throughput geometry optimisation can be performed by changing the `calc`:

In [ ]:
from aiida.plugins import CalculationFactory
GeomoptCalc = CalculationFactory("mlip.opt")

wg = get_ht_workgraph(
    calc=GeomoptCalc,
    folder=Path("../../tests/workflows/structures/"),
    calc_inputs=inputs,
    final_struct_key="final_structure",
)

In [ ]:
wg.run()

The full graph can be visualised:

In [ ]:
from aiida_workgraph.utils import generate_node_graph

generate_node_graph(wg.pk)

The final structures produced can then be accessed:

In [ ]:
wg.process.outputs.final_structures.H2O